In [ ]:
# Задание вытащить все вакансии СБЕРа с ХХ
# Вытащите все описания этих вакансий
# Создайте DataFrame только добавьте поле skills
# Переведите даты публикаций в datetime
# Постройте график опубликованных вакансий по датам
# Переведите даты в дни недели, и определите день недели, в который больше всего публикуют вакансий
# Найдите вакансии с использованием python
# Определите по полю skills какие навыки больше всего востребованы для этих вакансий, и
# Постройте график наиболее востребованных навыков

In [3]:
import requests
from tqdm.auto import tqdm
from collections import defaultdict
import pandas as pd

In [4]:
sber = '3529'
page = 1
num_per_page = 100
moscow = 1
url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={page}&per_page={num_per_page}&area={moscow}'

In [5]:
res = requests.get(url)

# res.json()

vacancies = res.json()
num_pages = vacancies.get('pages')
num_pages

12

In [10]:
vacancies.keys()

dict_keys(['items', 'found', 'pages', 'per_page', 'page', 'clusters', 'arguments', 'alternate_url'])

In [13]:
vacancies.get('items')

[{'id': '69034827',
  'premium': False,
  'name': 'Бизнес-аналитик',
  'department': {'id': '3529-3529-prof', 'name': 'Сбер для экспертов'},
  'has_test': False,
  'response_letter_required': False,
  'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
  'salary': None,
  'type': {'id': 'open', 'name': 'Открытая'},
  'address': None,
  'response_url': None,
  'sort_point_distance': None,
  'published_at': '2022-08-22T13:12:56+0300',
  'created_at': '2022-08-22T13:12:56+0300',
  'archived': False,
  'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=69034827',
  'insider_interview': None,
  'url': 'https://api.hh.ru/vacancies/69034827?host=hh.ru',
  'adv_response_url': 'https://api.hh.ru/vacancies/69034827/adv_response?host=hh.ru',
  'alternate_url': 'https://hh.ru/vacancy/69034827',
  'relations': [],
  'employer': {'id': '3529',
   'name': 'СБЕР',
   'url': 'https://api.hh.ru/employers/3529',
   'alternate_url': 'https://hh.ru/employer/35

In [ ]:
vacancies = res.json()
num_pages = vacancies.get('pages')
vacancy_ids = [el.get('id') for el in vacancies.get('items')]

In [ ]:
vacancies.keys()

In [ ]:
all_vacancy_ids= []
for i in tqdm(range(vacancies.get('pages'))):
    url = f'https://api.hh.ru/vacancies?employer_id={sber}&page={i}&per_page={num_per_page}&area={moscow}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    all_vacancy_ids.extend(vacancy_ids)

In [ ]:
len(all_vacancy_ids)

In [15]:
vac_id = 43326020
url = f'https://api.hh.ru/vacancies/{vac_id}'
res = requests.get(url)
vacancy = res.json()


In [16]:
vacancy

{'id': '43326020',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Специалист по экономической безопасности',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'experience': {'id': 'between1And3', 'name': 'От 1 года до 3 лет'},
 'schedule': {'id': 'fullDay', 'name': 'Полный день'},
 'employment': {'id': 'full', 'name': 'Полная занятость'},
 'department': {'id': '3529-3529-prof', 'name': 'Сбер для экспертов'},
 'contacts': None,
 'description': '<p><strong>Обязанности:</strong></p> <p> </p> <ul> <li>Поиск и анализ информации в отношении клиентов/контрагентов Банка для оценки уровня кредитных рисков, рисков деловой репутации, иных рисков сотрудничества с контрагентами, в т.ч. в рамках закупочных процедур, реализации строительных программ, б

In [ ]:
tabdict = defaultdict(list)
for vac_id in tqdm(all_vacancy_ids):
    url = f'https://api.hh.ru/vacancies/{vac_id}'
    res = requests.get(url, timeout=10)
    vacancy = res.json()
    tabdict['id'].append(vacancy.get('id'))
    tabdict['name'].append(vacancy.get('name'))
    tabdict['description'].append(vacancy.get('description'))
    tabdict['skills'].append([vacancy.get('key_skills')])
    tabdict['published_at'].append(vacancy.get('published_at'))


In [ ]:
sk = tabdict.get('skills')
tabdict['skills'] = [','.join([y.get('name') for y in x]) for x in sk]